## Pip installs

In [1]:
#TODO: add all pip install links OR create requirements.txt and pip install requirements
!pip install keras-tcn

  Using cached https://files.pythonhosted.org/packages/d4/54/d766211c883eb2bdb5dd4a1aa14e6a4240ae4206d1c799ee3ea2c1531d79/keras-tcn-3.1.1.tar.gz
  Could not find a version that satisfies the requirement numpy>=1.18.1 (from keras-tcn) (from versions: 1.3.0, 1.4.1, 1.5.0, 1.5.1, 1.6.0, 1.6.1, 1.6.2, 1.7.0, 1.7.1, 1.7.2, 1.8.0, 1.8.1, 1.8.2, 1.9.0, 1.9.1, 1.9.2, 1.9.3, 1.10.0.post2, 1.10.1, 1.10.2, 1.10.4, 1.11.0b3, 1.11.0rc1, 1.11.0rc2, 1.11.0, 1.11.1rc1, 1.11.1, 1.11.2rc1, 1.11.2, 1.11.3, 1.12.0b1, 1.12.0rc1, 1.12.0rc2, 1.12.0, 1.12.1rc1, 1.12.1, 1.13.0rc1, 1.13.0rc2, 1.13.0, 1.13.1, 1.13.3, 1.14.0rc1, 1.14.0, 1.14.1, 1.14.2, 1.14.3, 1.14.4, 1.14.5, 1.14.6, 1.15.0rc1, 1.15.0rc2, 1.15.0, 1.15.1, 1.15.2, 1.15.3, 1.15.4, 1.16.0rc1, 1.16.0rc2, 1.16.0, 1.16.1, 1.16.2, 1.16.3, 1.16.4, 1.16.5, 1.16.6)
No matching distribution found for numpy>=1.18.1 (from keras-tcn)


## Imports

In [2]:
#  IMPORTING LIBRARIES
import datetime as dt
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics
import sklearn
from sklearn import preprocessing

from statsmodels.tools.eval_measures import rmse

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.preprocessing.sequence import TimeseriesGenerator

from tcn import TCN

Using TensorFlow backend.


## Import Data

In [3]:
#  Example import, needs to be updated accordingly...
df = pd.read_csv("MockData.csv")

## Functions

In [4]:
def calculate_Pi(df):
    """Calculates the prediction result for a series, 1=true and 0=false"""
    
    result = (df['Actual'].shift(1) - df['Actual'])*(df['Forecast'].shift(1) - df['Forecast'])
    result.where(result <= 0, 1, inplace=True) # .where() returns if false.. hence reversed.
    result.where(result > 0, 0, inplace=True)
    return result

def calculate_hit_ratio(Pi):
    """Calculates the hit ratio for the prediction series Pi."""
    
    result = (Pi.sum()/ len(Pi))
    return result

def calculate_rmse(prediction, labels):
    """Function that calculates the Root Mean Square Error of predctions and labels"""
    
    rmse = np.sqrt(np.mean(prediction - labels) **2)
    return rmse

def calculate_r_squared(prediction, labels):
    """Function that calculates the R-squared error of predctions and labels"""
    
    correlation_matrix = np.corrcoef(prediction, labels)
    correlation_xy = correlation_matrix[0,1]
    r_squared = correlation_xy**2
    #r_squared = sklearn.metrics.r2_score(labels, prediction)
    return r_squared

## Prediction Models
### Linear Regression Model

In [5]:
def calculate_lrm_stats(_days, _df):
    """Function that calculates Hit Ratio, RMSE, R-squared values for forecasted 'days' forward"""
    
    forecast_days = _days
    _df['Label'] = _df['Close'].shift(-forecast_days)
    data = _df
    data.dropna(how="any", inplace=True)
    X = np.array(data.drop(['Label'],1))
    y = np.array(data['Label'])
    
    tss = TimeSeriesSplit(n_splits=5)
    for train_index, test_index in tss.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
    l = LinearRegression()
    l.fit(X_train, y_train.reshape(len(y_train)))
    
    forecast = l.predict(X_test)
    n = len(forecast)
    df_b_r = pd.DataFrame({
        'DateTime':data.index[-n:],
        'Forecast':forecast,
        'Actual':y_test})
    df_b_r.set_index('DateTime', inplace=True)
    
    Pi = calculate_Pi(df_b_r)
    hit_r = calculate_hit_ratio(Pi)
    rmse = calculate_rmse(df_b_r.Forecast, df_b_r.Actual)
    train_r2 = l.score(X_train, y_train)
    test_r2 = l.score(X_test, y_test)
    
    return (forecast_days, hit_r, rmse, train_r2, test_r2, df_b_r, Pi)

### LSTM Model

In [6]:
def calculate_lstm_stats(_days, _df, _epochs, _n_features):
    """Function that calculates LSTM Model statistics"""
    
    _batch_size = 16
    forecast_days = _days
    _df['Label'] = _df['Close'].shift(-forecast_days)
    data = _df
    data.dropna(how="any", inplace=True)
    X = np.array(data.drop(['Label'],1))
    y = np.array(data['Label'])
    
    tss = TimeSeriesSplit(n_splits=5)
    for train_index, test_index in tss.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
    n_input = 14
    n_features = _n_features
    
    generator_train = TimeseriesGenerator(X_train, y_train, length = n_input, batch_size=_batch_size)
    generator_test = TimeseriesGenerator(X_test, y_test, length = n_input, batch_size=_batch_size)
    
    model = Sequential()
    model.add(LSTM(120, activation='tanh', input_shape=(n_input, n_features)))
    model.add(Dropout(0.15)) # prevents overfitting
    model.add(Dense(50))
    model.add(Dense(40))
    model.add(Dense(30))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit_generator(generator_train, epochs=_epochs)
    
    lstm_prediction_test = model.predict(generator_test)
    lstm_prediction_train = model.predict(generator_train)
    
    n = len(lstm_prediction_test)
    m = len(lstm_prediction_train)
    
    df_lstm = pd.DataFrame(
        {'DateTime':data.index[-n:],
         'Forecast':lstm_prediction_test.reshape(n,), 
         'Actual':y_test[-n:]})
    df_lstm.set_index('DateTime', inplace=True)
    
    Pi = calculate_Pi(df_lstm)
    hit_r = calculate_hit_ratio(Pi)
    rmse = calculate_rmse(lstm_prediction_test, y_test[-n:])
    train_r2 = calculate_r_squared(lstm_prediction_train.reshape(m,), y_train[-m:])
    test_r2 = calculate_r_squared(lstm_prediction_test.reshape(n,), y_test[-n:])
    
    return (forecast_days, hit_r, rmse, train_r2, test_r2, df_lstm, Pi)

## Refactored Models
Models to refactor:
- Linear Regression
- LSTM
- TCNN
- SVR
- SVC

In [ ]:
def linear_regression(data, settings):
    """Creates a linear regression model and predictions.
    
    Args:
        data: pandas.DataFrame.
        settings: Dictionary object containing settings parameters.
    Returns:
        A dictionary containing the linear regression model and predictions.
    """
    
    #  VARIABLES
    x_train = data['train']['features']
    y_train = data['train']['labels']
    x_test = data['test']['features']
    scaler = data['scaler']
    
    #  INSTANTIATE MODEL
    model = linear.fit(x_train, y_train)
    
    #  PREDICTIONS
    predictions = model.predict(x_test)
    
    #  DENORMALIZED PREDICTIONS FOR ACTUAL PREDICTION VALUES
    denormalized_predictions = scaler.inverse_transform(predictions)
    
    return {
        'model': model,
        'predictions': denormalized_predictions
    }

In [ ]:
def long_short_term_memory(data, settings):
    """Creates a Long short-term memory model (LSTM) and predictions.
    
    Args:
        data: pandas.DataFrame.
        settings: Dictionary object containing settings parameters.
    Returns:
        A dictionary containing the LSTM model and predictions.
    """
    
    #  VARIABLES
    
    #  INSTANTIATE MODEL
    model = Sequential()
    
    
    denormalized_predictions = ""
    
    return {
        'model': model,
        'predictions': denormalized_predictions
    }

## Test Implementation

In [7]:
df = df.set_index('DateTime')
df_lstm = df
df_lrm = df

In [8]:
#  Linear Regression Model, forecast 1 day forward
LRM_1 = calculate_lrm_stats(1, df_lrm)

In [9]:
#  LSTM Model, forecast 1 day forward
LSTM_1 = calculate_lstm_stats(_days=1, _df=df_lstm, _epochs=5, _n_features=5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Epoch 1/5
161/161 [==============================] - 3s 16ms/step - loss: 0.0123
Epoch 2/5
161/161 [==============================] - 2s 14ms/step - loss: 0.0019
Epoch 3/5
161/161 [==============================] - 3s 18ms/step - loss: 0.0033
Epoch 4/5
161/161 [==============================] - 2s 15ms/step - loss: 0.0017
Epoch 5/5
161/161 [==============================] - 2s 13ms/step - loss: 0.0031


In [10]:
LRM_1[-2].to_csv(r'LRM_results.csv')
LSTM_1[-2].to_csv(r'LSTM_results.csv')